# Домашнее задание 1

In [1]:
import pandas as pd
from datetime import datetime

### Загружаем данные

In [2]:
column_names = ['id', 'tdate', 'tname', 'ttext', 'ttype', 'trep', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount', 'unknown']
positive = pd.read_csv('D:/YandexDisk/DataScientist/DataSets/twitter_/positive.csv', delimiter=";", header=None, names=column_names )
negative = pd.read_csv('D:/YandexDisk/DataScientist/DataSets/twitter_/negative.csv', delimiter=";", header=None, names=column_names )

#### Сцепляем выборки

In [3]:
full = positive.append(negative).sample(frac=1)

In [4]:
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226834 entries, 107898 to 107515
Data columns (total 12 columns):
id           226834 non-null int64
tdate        226834 non-null int64
tname        226834 non-null object
ttext        226834 non-null object
ttype        226834 non-null int64
trep         226834 non-null int64
tfav         226834 non-null int64
tstcount     226834 non-null int64
tfol         226834 non-null int64
tfrien       226834 non-null int64
listcount    226834 non-null int64
unknown      226834 non-null int64
dtypes: int64(10), object(2)
memory usage: 22.5+ MB


In [6]:
full.head()

,id,tdate,tname,ttext,ttype,trep,tfav,tstcount,tfol,tfrien,listcount,unknown
107898,424276542370242560,1389990385,RudikNatalia,@BTKris89 будешь потом рассказывать мне про ко...,-1,0,0,0,3147,47,136,0
39733,413895335564812288,1387515312,BydawayEugene,Ну почему вова не идет завтра на уроки без нег...,-1,0,0,0,472,280,328,0
17173,409368204113641473,1386435960,egormaloi,Епта проодолжаем праздник!!.ебашим на гоорку к...,1,0,0,0,791,20,19,0
55579,410083670960386048,1386606540,bestanimefka,"Музыка сегодня будет разной, на столько разной...",1,0,0,0,1972,65,1,1
2998,409044286005465088,1386358732,MysteriousThe,"@grapes2705 пф, АЗАХАХАХ, НЕ НА ТУ НАРВАЛАСЬ, ...",1,0,0,0,9446,102,45,1


### Разбиваем на train и test

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(full['ttext'], full['ttype'], test_size=0.2)

### Загружаем стоп-слова

In [11]:
stop_words = [x.strip() for x in open('stop.txt','r', encoding='utf-8').read().split('\n')]
stop_words

['а',
 'будем',
 'будет',
 'будете',
 'будешь',
 'буду',
 'будут',
 'будучи',
 'будь',
 'будьте',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вами',
 'вас',
 'весь',
 'во',
 'вот',
 'все',
 'всё',
 'всего',
 'всей',
 'всем',
 'всём',
 'всеми',
 'всему',
 'всех',
 'всею',
 'всея',
 'всю',
 'вся',
 'вы',
 'да',
 'для',
 'до',
 'его',
 'едим',
 'едят',
 'ее',
 'её',
 'ей',
 'ел',
 'ела',
 'ем',
 'ему',
 'емъ',
 'если',
 'ест',
 'есть',
 'ешь',
 'еще',
 'ещё',
 'ею',
 'же',
 'за',
 'и',
 'из',
 'или',
 'им',
 'ими',
 'имъ',
 'их',
 'к',
 'как',
 'кем',
 'ко',
 'когда',
 'кого',
 'ком',
 'кому',
 'комья',
 'которая',
 'которого',
 'которое',
 'которой',
 'котором',
 'которому',
 'которою',
 'которую',
 'которые',
 'который',
 'которым',
 'которыми',
 'которых',
 'кто',
 'меня',
 'мне',
 'мной',
 'мною',
 'мог',
 'моги',
 'могите',
 'могла',
 'могли',
 'могло',
 'могу',
 'могут',
 'мое',
 'моё',
 'моего',
 'моей',
 'моем',
 'моём',
 'моему',
 'моею',
 'можем',
 'может'

### Готовим pipeline

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

pipe = Pipeline([('vectorizer', TfidfVectorizer(stop_words=stop_words)),
                ('classifier', MultinomialNB())])

### Для проверки разных токенов сделаем кросс-валидацию

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

kfold = KFold(n_splits=4,shuffle=True,random_state=42)

param_grid = {'vectorizer__token_pattern': ['(?u)\\b[А-Яа-я0-9\-][А-Яа-я0-9\-]+\\b', '(?u)\\b\\w\\w+\\b', '(?u)\\b[А-Яа-я][А-Яа-я]+\\b']}

grid = GridSearchCV(pipe, param_grid, scoring=make_scorer(f1_score), cv=kfold)
grid = grid.fit(X_train,y_train)


In [14]:
tokens = grid.cv_results_['param_vectorizer__token_pattern']
f1s = grid.cv_results_['mean_test_score']
for i in range(0, len(tokens)):
    print("For token '", tokens[i], "' result is ", f1s[i])

For token ' (?u)\b[А-Яа-я0-9\-][А-Яа-я0-9\-]+\b ' result is  0.723001517551
For token ' (?u)\b\w\w+\b ' result is  0.741970286575
For token ' (?u)\b[А-Яа-я][А-Яа-я]+\b ' result is  0.722015220014


### Победил стандартный токен. Используем его

In [15]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...      vocabulary=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [16]:
predicted = pipe.predict(X_test)

In [17]:
import sklearn.metrics
print(sklearn.metrics.classification_report(y_test, predicted))

             precision    recall  f1-score   support

         -1       0.74      0.77      0.76     22501
          1       0.77      0.74      0.75     22866

avg / total       0.75      0.75      0.75     45367



In [18]:
sklearn.metrics.confusion_matrix(y_test, predicted)

array([[17321,  5180],
       [ 5994, 16872]])